In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris

In [2]:
iris = load_iris()
data = pd.DataFrame(iris.data, columns=iris.feature_names)
data['target'] = iris.target

In [ ]:
import torch.nn as nn
m = nn.Linear(20, 30, ac)
input = torch.randn(128, 20)
output = m(input)
print(output.size())

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 定義 smish 函數
def smish(x):
    return x * torch.tanh(torch.log(1 + torch.sigmoid(x)))

class GatedLinearUnit(nn.Module):
    def __init__(self, units):
        super(GatedLinearUnit, self).__init__()
        self.linear = nn.Linear(units)
        self.sigmoid = nn.Sequential(nn.Linear(units), nn.Sigmoid())

    def forward(self, inputs):
        return self.linear(inputs) * self.sigmoid(inputs)

class GatedResidualNetwork(nn.Module):
    def __init__(self, units, dropout_rate):
        super(GatedResidualNetwork, self).__init__()
        self.units = units
        self.dropout_rate = dropout_rate
        self.relu_dense = nn.Sequential(nn.Linear(units), smish())
        self.linear_dense = nn.Linear(units)
        self.dropout = nn.Dropout(dropout_rate)
        self.gated_linear_unit = GatedLinearUnit(units)
        self.layer_norm = nn.LayerNorm()
        self.project = nn.Linear(units)

    def forward(self, inputs):
        x = self.relu_dense(inputs)
        x = self.linear_dense(x)
        x = self.dropout(x)
        if inputs.shape[-1] != self.units:
            inputs = self.project(inputs)
        x = inputs + self.gated_linear_unit(x)
        x = self.layer_norm(x)
        return x

class VariableSelection(nn.Module):
    def __init__(self, num_features, units, dropout_rate):
        super(VariableSelection, self).__init__()
        self.grns = nn.ModuleList()
        # Create a GRN for each feature independently
        for idx in range(num_features):
            grn = GatedResidualNetwork(units, dropout_rate)
            self.grns.append(grn)
        # Create a GRN for the concatenation of all the features
        self.grn_concat = GatedResidualNetwork(units, dropout_rate)
        self.softmax = nn.Sequential(nn.Linear(units), nn.Softmax())
        self.num_features = num_features
        self.units = units
        self.dropout_rate = dropout_rate


    def forward(self, inputs):
        v = torch.cat(inputs, dim=-1)
        v = self.grn_concat(v)
        v = F.softmax(v, dim=-1)

        x = []
        for idx, input_ in enumerate(inputs):
            x.append(self.grns[idx](input_))
        x = torch.stack(x, dim=1)

        outputs = torch.squeeze(torch.matmul(v.unsqueeze(2), x), dim=2)
        #torch.squeeze(torch.bmm(v.unsqueeze(2), x), dim=2)
        return outputs

class VariableSelectionFlow(nn.Module):
    def __init__(self, num_features, units, dropout_rate, dense_units=None):
        super(VariableSelectionFlow, self).__init__()
        self.variableselection = VariableSelection(num_features, units, dropout_rate)
        self.num_features = num_features
        self.units = units
        self.dropout_rate = dropout_rate
        self.dense_units = dense_units
        if dense_units:
            self.dense_list = nn.ModuleList([
                nn.Linear(dense_units, dense_units) for _ in range(num_features)
            ])

    def forward(self, inputs):
        split_input = torch.chunk(inputs, self.num_features, dim=-1)
        if self.dense_units:
            l = [self.dense_list[i](split_input[i]) for i in range(self.num_features)]
        else:
            l = split_input
        return self.variableselection(l)


In [5]:
from sklearn.model_selection import train_test_split

trainX, testX, trainy, testy = train_test_split(data.drop(columns=['target'], axis=1), data[['target']], random_state=42)

In [8]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

# 定義自定義的 VariableSelectionFlow 層（已經在前面的回答中定義）
# ... 請在這裡添加前面的 VariableSelectionFlow 定義 ...

class CustomModel(nn.Module):
    def __init__(self, units_1, drop_1, units_2, drop_2, units_3, drop_3, dense_units):
        super(CustomModel, self).__init__()
        self.features_1 = VariableSelectionFlow(4, units_1, drop_1, dense_units=dense_units)
        self.features_2 = VariableSelectionFlow(units_1, units_2, drop_2)
        self.features_3 = VariableSelectionFlow(units_2, units_3, drop_3)
        self.dense = nn.Linear(units_3, 3)

    def forward(self, x):
        print(x.shape)
        x = self.features_1(x)
        x = self.features_2(x)
        x = self.features_3(x)
        x = self.dense(x)
        return x

batch_size = 32
units_1 = 32
drop_1 = 0.75
dense_units = 8
units_2 = 16
drop_2 = 0.5
units_3 = 8
drop_3 = 0.25

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=722)
X = torch.tensor(trainX.values, dtype=torch.float32)
y_enc = LabelEncoder().fit_transform(trainy)
y_label = torch.tensor(np.eye(3)[y_enc], dtype=torch.float32)
blls = []

for n, (train_idx, val_idx) in enumerate(cv.split(trainX, trainy)):
    for k in range(3):
        print(f'______fold {n+1}______, ________repeat {k+1}__________')
        
        model = CustomModel(units_1, drop_1, units_2, drop_2, units_3, drop_3, dense_units)
        
        optimizer = optim.Adam(model.parameters(), lr=1e-3, eps=1e-7)
        loss_fn = nn.CrossEntropyLoss()

        lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.95, patience=1, verbose=True)
        es = EarlyStopping(monitor='val_loss', mode='min', patience=25, verbose=False)
        
        train_data = TensorDataset(X[train_idx], y_label[train_idx])
        train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
        val_data = TensorDataset(X[val_idx], y_label[val_idx])
        val_loader = DataLoader(val_data, batch_size=batch_size)

        for epoch in range(20):
            model.train()
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = loss_fn(outputs, torch.argmax(labels, dim=1))
                loss.backward()
                optimizer.step()

            model.eval()
            val_losses = []
            with torch.no_grad():
                for inputs, labels in val_loader:
                    outputs = model(inputs)
                    val_loss = loss_fn(outputs, torch.argmax(labels, dim=1))
                    val_losses.append(val_loss.item())
            avg_val_loss = np.mean(val_losses)
            lr_scheduler.step(avg_val_loss)
            print(f'Epoch [{epoch+1}/20], Validation Loss: {avg_val_loss:.4f}')

        bll = nn.CrossEntropyLoss()(model(X[val_idx]), torch.argmax(y_label[val_idx], dim=1))
        blls.append(bll)
        print(f'BLL: {bll:.4f}')

        torch.save(model.state_dict(), f'models_weights/mod_f{n}_r{k}.pth')

print(np.mean(blls))

/Users/yukaisun/opt/anaconda3/envs/tabpfn/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


______fold 1______, ________repeat 1__________


TypeError: __init__() missing 1 required positional argument: 'out_features'

In [18]:
for i, j in train_loader:
    print(i.shape)

torch.Size([32, 4])
torch.Size([32, 4])
torch.Size([25, 4])


In [13]:
train_data.to_numpy()

AttributeError: 'TensorDataset' object has no attribute 'to_numpy'